![imagen](images/sql_murder.png)

# Ejercicio SQL Murder
Para este ejercicio vas a ponerte la gabardina y el sombrero ya que tendrás que investigar un asesinato en *SQL City*. 

Aplicarás los conecptos básicos de SQL y aprenderás a manejarte con un modelo de datos, mientras resuelves un caso de asesinato :)

Este espectacular juego está creado por **Joon Park y Cathy He**, y podrás encontrar todos los detalles en [su web](http://mystery.knightlab.com)

Para realizar el ejercicio tienes dos opciones, o bien desde la propia web, donde encontrarás un intérprete de sentencias SQL interactivo. Si usas esta opción te recomiendo que vayas apuntando las queries y sus outputs. O la otra opción es [acudir a su GitHub](https://github.com/NUKnightLab/sql-mysteries), descargarte la base de datos y realizar las queries. Probablemente la primera opción te sea más cómoda


### Enunciado
A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **murder** that occurred sometime on **Jan.15, 2018** and that it took place in **SQL City**. Start by retrieving the corresponding crime scene report from the police department’s database.


### Modelo de datos
![imagen](images/sql_game_schemma.png)

¡Suerte Sherlock!

# SOLUCIÓN

## 1. Acudir a la escena del crimen


```SQL
SELECT *
FROM crime_scene_report
WHERE type = 'murder' AND city = 'SQL City' AND date = '20180115';
```
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


## 2. Hay que preguntar a los dos testigos

### 2.1 Testigo 1

```SQL
SELECT name, max(address_number) as "Last House"
FROM person
WHERE address_street_name = 'Northwestern Dr';
```
id -> 14887
name -> Morty Schapiro
address_street_name -> Northwestern Dr


### 2.2 Testigo 2

```SQL
SELECT *
FROM person
WHERE address_street_name = 'Franklin Ave' and name like '%Annabel%';
```
id -> 16371
name -> Annabel Miller
address_street_name -> Franklin Ave

## 3. Ver qué dijeron en las entrevistas con la policía

### 3.1 Testimonio testigo 1

```SQL
SELECT *
FROM interview
WHERE person_id = 14887;
```

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".


### 3.1 Testimonio testigo 2
```SQL
SELECT *
FROM interview
WHERE person_id = 16371;
```

I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


## 4. Pistas testimonio

## 4.1 Pistas testimonio testimonio 1
Miembros del gimasio que empiezan por "48Z" y son gold members

```SQL
SELECT *
FROM get_fit_now_member
WHERE id LIKE '48Z%' AND membership_Status = 'gold';
```
Tengo dos posibles opciones: Joe Germuska y Jeremy Bowers

Buscamos por matricula
```SQL
SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%' AND gender = "male";
```

Nos salen dos posibles coches, cuyo id de licencia de conducir es 423327 o 664760


## 4.2 Pistas testimonio testimonio 2
Ambos fueron al gimnasio el 9 de enero, y ademas sabemos del 4.1, que tiene que ser o '48Z7A' o '48Z55'

```SQL
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = '20180109' AND membership_id IN ('48Z7A', '48Z55')
```

No me da más información

## 5. Buscamos en la tabla de personas, las obtenidas con el primer testigo
Queremos sacar su licencia de conducir, para cruzar con la informacion del coche del testigo 

```SQL
SELECT *
FROM person
WHERE name IN ('Joe Germuska', 'Jeremy Bowers');
```

Tienen las siguientes licencias de conducir:
Joe Germuska: 173289
Jeremy Bowers: 423327

Por la matricula, sabemos que tenemos dos posibles licencias de conducir: 423327 o 664760

Vemos que una de ellas coincide con unos d elos sospechosos, por lo que ha sido Jeremy Bowers!!!

## 6. Probamos si es ese
I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.

Efectivamente es Jeremy Bowers. Hay que buscar ahora a una mujer

## 7. Pistas de la mujer rica

### 7.1 Buscamos los datos del concierto
```SQL
SELECT COUNT(person_id) AS conteo, person_id
FROM facebook_event_checkin
WHERE event_name LIKE '%SQL Symphony Concert%' AND date LIKE '201712%'
GROUP BY person_id
ORDER BY conteo DESC;
```

Hay dos personas que acudieron a ese concierto 3 veces en esa fecha: 24556 o 99716


### 7.2 Busco si estas personas tienen carnet de conducir.
Ya tengo a la persona. Me dicen que lleva un Tesla, por lo que comprobamos si tiene carnet de conducir

```SQL
SELECT person.id, person.name, person.license_id
FROM person
INNER JOIN drivers_license ON person.license_id = drivers_license.id
WHERE person.id IN (24556, 99716)
```

Da como resultado Miranda Priestly!



## OTRAS QUERIES

Siguiendo la pista de que es una mujer con dinero...esta no sirve de mucho...
```SQL
SELECT a.ssn, a.annual_income, b.id, b.name
FROM income as a
LEFT JOIN person as b
on a.ssn = b.ssn
ORDER BY a.annual_income DESC;
```